## 1. Setup & Installation

In [ ]:
# Install dependencies
!pip install -q torch torchvision numpy matplotlib tqdm

In [ ]:
# Clone repository
!git clone https://github.com/QuocKhanhLuong/FourierNetwork.git
%cd FourierNetwork

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️ Using device: {device}")
if device == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Import Models

In [ ]:
# Import our models
from monogenic import EnergyMap, MonogenicSignal, BoundaryDetector
from gabor_implicit import GaborBasis, GaborNet, ImplicitSegmentationHead
from egm_net import EGMNet, EGMNetLite
from spectral_mamba import SpectralVMUNet

print("✅ All modules imported successfully!")

## 🏥 REAL DATA TRAINING

This section trains on real medical imaging datasets. Choose from:

| Dataset | Modality | Classes | Description |
|---------|----------|---------|-------------|
| **ACDC** | Cardiac MRI | 4 | LV, RV, Myocardium + Background |
| **BraTS21** | Brain MRI | 4 | Tumor core, Enhancing, Edema + Background |
| **M&Ms** | Cardiac MRI | 4 | Multi-center, Multi-vendor cardiac segmentation |

**Your Google Drive Links:**
- ACDC: `1EelzBVjIoDQ4uzt0_2JzmF_PuUHsD93e`
- BraTS21: `1m7b5u_6cEj9PbqzZQDNoDonnXIowBCY2`
- M&Ms: `1DpW8ucYE17Tj8iMlsAev_yM6TaZZmWYj`

In [ ]:
# Mount Google Drive and select dataset
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# ============================================================
# 🔧 CONFIGURE YOUR DATASET HERE
# ============================================================
DATASET = 'ACDC'  # Options: 'ACDC', 'BraTS21', 'MnM'

# Google Drive folder IDs (from your links)
DRIVE_FOLDERS = {
    'ACDC': '1EelzBVjIoDQ4uzt0_2JzmF_PuUHsD93e',
    'BraTS21': '1m7b5u_6cEj9PbqzZQDNoDonnXIowBCY2',
    'MnM': '1DpW8ucYE17Tj8iMlsAev_yM6TaZZmWYj'
}

# Dataset configurations
DATASET_CONFIG = {
    'ACDC': {
        'num_classes': 4,  # Background, RV, Myo, LV
        'class_names': ['Background', 'RV', 'Myocardium', 'LV'],
        'in_channels': 1,
        'img_size': 256,
        'drive_path': '/content/drive/MyDrive/ACDC'  # Adjust if different
    },
    'BraTS21': {
        'num_classes': 4,  # Background, NCR/NET, ED, ET
        'class_names': ['Background', 'NCR/NET', 'Edema', 'Enhancing'],
        'in_channels': 4,  # T1, T1ce, T2, FLAIR
        'img_size': 256,
        'drive_path': '/content/drive/MyDrive/BraTS21'
    },
    'MnM': {
        'num_classes': 4,  # Background, RV, Myo, LV
        'class_names': ['Background', 'RV', 'Myocardium', 'LV'],
        'in_channels': 1,
        'img_size': 256,
        'drive_path': '/content/drive/MyDrive/MnM'
    }
}

# Get current config
config_data = DATASET_CONFIG[DATASET]
print(f"📊 Selected Dataset: {DATASET}")
print(f"   Classes: {config_data['num_classes']} - {config_data['class_names']}")
print(f"   Input channels: {config_data['in_channels']}")
print(f"   Image size: {config_data['img_size']}×{config_data['img_size']}")

In [ ]:
# Check dataset location and structure
import glob

# Try to find data in Drive
drive_path = config_data['drive_path']

if os.path.exists(drive_path):
    print(f"✅ Found dataset at: {drive_path}")
    print("\n📁 Directory structure:")
    for item in os.listdir(drive_path)[:10]:
        item_path = os.path.join(drive_path, item)
        if os.path.isdir(item_path):
            print(f"   📂 {item}/")
            sub_items = os.listdir(item_path)[:3]
            for sub in sub_items:
                print(f"      └── {sub}")
        else:
            print(f"   📄 {item}")
else:
    print(f"⚠️ Dataset not found at: {drive_path}")
    print("\n🔧 Please update 'drive_path' in DATASET_CONFIG above")
    print("   Or upload your data to the expected location")
    
    # Try alternative locations
    alt_paths = [
        f'/content/drive/MyDrive/{DATASET}',
        f'/content/drive/MyDrive/Datasets/{DATASET}',
        f'/content/drive/MyDrive/Medical/{DATASET}'
    ]
    for alt in alt_paths:
        if os.path.exists(alt):
            print(f"\n✅ Found at alternative location: {alt}")
            drive_path = alt
            break

In [ ]:
# Dataset Classes for ACDC, BraTS21, M&Ms
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import glob
import h5py
import nibabel as nib


class ACDCDataset(Dataset):
    """
    ACDC Cardiac MRI Dataset
    Classes: Background (0), RV (1), Myocardium (2), LV (3)
    """
    def __init__(self, data_dir, img_size=256, split='train', transform=None):
        self.img_size = img_size
        self.split = split
        self.transform = transform
        
        # Find all patient folders or npz files
        self.samples = []
        
        # Check for npz format (preprocessed)
        npz_files = sorted(glob.glob(os.path.join(data_dir, f'{split}/*.npz')))
        if len(npz_files) > 0:
            self.samples = npz_files
            self.format = 'npz'
            print(f"   Found {len(self.samples)} NPZ files")
        else:
            # Check for h5 format
            h5_files = sorted(glob.glob(os.path.join(data_dir, f'{split}/*.h5')))
            if len(h5_files) > 0:
                self.samples = h5_files
                self.format = 'h5'
                print(f"   Found {len(self.samples)} H5 files")
            else:
                # Check for NIfTI format (raw ACDC)
                patient_dirs = sorted(glob.glob(os.path.join(data_dir, 'patient*')))
                for patient_dir in patient_dirs:
                    # Find frame images and ground truth
                    gt_files = glob.glob(os.path.join(patient_dir, '*_gt.nii.gz'))
                    for gt_file in gt_files:
                        img_file = gt_file.replace('_gt.nii.gz', '.nii.gz')
                        if os.path.exists(img_file):
                            self.samples.append((img_file, gt_file))
                self.format = 'nifti'
                print(f"   Found {len(self.samples)} NIfTI pairs")
                
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        if self.format == 'npz':
            data = np.load(self.samples[idx])
            img = data['image'].astype(np.float32)
            mask = data['label'].astype(np.int64)
            
        elif self.format == 'h5':
            with h5py.File(self.samples[idx], 'r') as f:
                img = f['image'][()].astype(np.float32)
                mask = f['label'][()].astype(np.int64)
                
        elif self.format == 'nifti':
            img_file, gt_file = self.samples[idx]
            img_nii = nib.load(img_file)
            gt_nii = nib.load(gt_file)
            
            img = img_nii.get_fdata().astype(np.float32)
            mask = gt_nii.get_fdata().astype(np.int64)
            
            # Take middle slice if 3D
            if len(img.shape) == 3:
                mid_slice = img.shape[2] // 2
                img = img[:, :, mid_slice]
                mask = mask[:, :, mid_slice]
        
        # Normalize image
        img = (img - img.min()) / (img.max() - img.min() + 1e-8)
        
        # Resize if needed
        if img.shape[0] != self.img_size or img.shape[1] != self.img_size:
            img = np.array(Image.fromarray(img).resize((self.img_size, self.img_size), Image.BILINEAR))
            mask = np.array(Image.fromarray(mask.astype(np.uint8)).resize((self.img_size, self.img_size), Image.NEAREST))
        
        # Convert to tensors
        img = torch.from_numpy(img).unsqueeze(0).float()
        mask = torch.from_numpy(mask).long()
        
        return img, mask


class BraTS21Dataset(Dataset):
    """
    BraTS 2021 Brain Tumor Segmentation Dataset
    Modalities: T1, T1ce, T2, FLAIR (4 channels)
    Classes: Background (0), NCR/NET (1), Edema (2), Enhancing Tumor (3)
    """
    def __init__(self, data_dir, img_size=256, split='train', transform=None):
        self.img_size = img_size
        self.split = split
        self.transform = transform
        self.samples = []
        
        # Check for npz format
        npz_files = sorted(glob.glob(os.path.join(data_dir, f'{split}/*.npz')))
        if len(npz_files) > 0:
            self.samples = npz_files
            self.format = 'npz'
            print(f"   Found {len(self.samples)} NPZ files")
        else:
            # Check for NIfTI format
            patient_dirs = sorted(glob.glob(os.path.join(data_dir, 'BraTS*')))
            for patient_dir in patient_dirs:
                t1_file = glob.glob(os.path.join(patient_dir, '*_t1.nii.gz'))
                t1ce_file = glob.glob(os.path.join(patient_dir, '*_t1ce.nii.gz'))
                t2_file = glob.glob(os.path.join(patient_dir, '*_t2.nii.gz'))
                flair_file = glob.glob(os.path.join(patient_dir, '*_flair.nii.gz'))
                seg_file = glob.glob(os.path.join(patient_dir, '*_seg.nii.gz'))
                
                if all([t1_file, t1ce_file, t2_file, flair_file, seg_file]):
                    self.samples.append({
                        't1': t1_file[0],
                        't1ce': t1ce_file[0],
                        't2': t2_file[0],
                        'flair': flair_file[0],
                        'seg': seg_file[0]
                    })
            self.format = 'nifti'
            print(f"   Found {len(self.samples)} patient folders")
            
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        if self.format == 'npz':
            data = np.load(self.samples[idx])
            img = data['image'].astype(np.float32)  # (4, H, W) or (H, W, 4)
            mask = data['label'].astype(np.int64)
            
            if len(img.shape) == 3 and img.shape[-1] == 4:
                img = img.transpose(2, 0, 1)  # (H, W, 4) -> (4, H, W)
                
        elif self.format == 'nifti':
            sample = self.samples[idx]
            
            # Load all modalities
            t1 = nib.load(sample['t1']).get_fdata()
            t1ce = nib.load(sample['t1ce']).get_fdata()
            t2 = nib.load(sample['t2']).get_fdata()
            flair = nib.load(sample['flair']).get_fdata()
            mask = nib.load(sample['seg']).get_fdata()
            
            # Take middle slice
            mid_slice = t1.shape[2] // 2
            t1 = t1[:, :, mid_slice]
            t1ce = t1ce[:, :, mid_slice]
            t2 = t2[:, :, mid_slice]
            flair = flair[:, :, mid_slice]
            mask = mask[:, :, mid_slice]
            
            # Stack modalities
            img = np.stack([t1, t1ce, t2, flair], axis=0).astype(np.float32)
            mask = mask.astype(np.int64)
        
        # Normalize each channel
        for c in range(img.shape[0]):
            ch = img[c]
            img[c] = (ch - ch.min()) / (ch.max() - ch.min() + 1e-8)
        
        # Resize if needed
        if img.shape[1] != self.img_size:
            new_img = np.zeros((img.shape[0], self.img_size, self.img_size), dtype=np.float32)
            for c in range(img.shape[0]):
                new_img[c] = np.array(Image.fromarray(img[c]).resize((self.img_size, self.img_size), Image.BILINEAR))
            img = new_img
            mask = np.array(Image.fromarray(mask.astype(np.uint8)).resize((self.img_size, self.img_size), Image.NEAREST))
        
        # Remap BraTS labels: 0->0, 1->1, 2->2, 4->3
        mask[mask == 4] = 3
        
        img = torch.from_numpy(img).float()
        mask = torch.from_numpy(mask).long()
        
        return img, mask


class MnMDataset(Dataset):
    """
    M&Ms (Multi-Centre, Multi-Vendor) Cardiac MRI Dataset
    Classes: Background (0), RV (1), Myocardium (2), LV (3)
    """
    def __init__(self, data_dir, img_size=256, split='train', transform=None):
        self.img_size = img_size
        self.split = split
        self.transform = transform
        self.samples = []
        
        # Check for npz format
        npz_files = sorted(glob.glob(os.path.join(data_dir, f'{split}/*.npz')))
        if len(npz_files) > 0:
            self.samples = npz_files
            self.format = 'npz'
            print(f"   Found {len(self.samples)} NPZ files")
        else:
            # Check for h5 format
            h5_files = sorted(glob.glob(os.path.join(data_dir, f'{split}/*.h5')))
            if len(h5_files) > 0:
                self.samples = h5_files
                self.format = 'h5'
                print(f"   Found {len(self.samples)} H5 files")
            else:
                # Check for NIfTI (raw M&Ms)
                patient_dirs = sorted(glob.glob(os.path.join(data_dir, '*/sa.nii.gz')))
                for sa_file in patient_dirs:
                    gt_file = sa_file.replace('sa.nii.gz', 'sa_gt.nii.gz')
                    if os.path.exists(gt_file):
                        self.samples.append((sa_file, gt_file))
                self.format = 'nifti'
                print(f"   Found {len(self.samples)} NIfTI pairs")
                
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        if self.format == 'npz':
            data = np.load(self.samples[idx])
            img = data['image'].astype(np.float32)
            mask = data['label'].astype(np.int64)
            
        elif self.format == 'h5':
            with h5py.File(self.samples[idx], 'r') as f:
                img = f['image'][()].astype(np.float32)
                mask = f['label'][()].astype(np.int64)
                
        elif self.format == 'nifti':
            img_file, gt_file = self.samples[idx]
            img = nib.load(img_file).get_fdata().astype(np.float32)
            mask = nib.load(gt_file).get_fdata().astype(np.int64)
            
            # Take middle slice
            if len(img.shape) >= 3:
                mid_slice = img.shape[2] // 2
                img = img[:, :, mid_slice]
                mask = mask[:, :, mid_slice]
        
        # Normalize
        img = (img - img.min()) / (img.max() - img.min() + 1e-8)
        
        # Resize
        if img.shape[0] != self.img_size:
            img = np.array(Image.fromarray(img).resize((self.img_size, self.img_size), Image.BILINEAR))
            mask = np.array(Image.fromarray(mask.astype(np.uint8)).resize((self.img_size, self.img_size), Image.NEAREST))
        
        img = torch.from_numpy(img).unsqueeze(0).float()
        mask = torch.from_numpy(mask).long()
        
        return img, mask


# Create dataset based on selection
print(f"\n📊 Loading {DATASET} dataset...")

if DATASET == 'ACDC':
    train_dataset = ACDCDataset(drive_path, img_size=config_data['img_size'], split='train')
    val_dataset = ACDCDataset(drive_path, img_size=config_data['img_size'], split='val')
elif DATASET == 'BraTS21':
    train_dataset = BraTS21Dataset(drive_path, img_size=config_data['img_size'], split='train')
    val_dataset = BraTS21Dataset(drive_path, img_size=config_data['img_size'], split='val')
elif DATASET == 'MnM':
    train_dataset = MnMDataset(drive_path, img_size=config_data['img_size'], split='train')
    val_dataset = MnMDataset(drive_path, img_size=config_data['img_size'], split='val')

# Fallback to synthetic if no data found
if len(train_dataset) == 0:
    print("⚠️ No data found! Using synthetic dataset for testing...")
    
    class SyntheticMedicalDataset(Dataset):
        def __init__(self, num_samples=500, img_size=256, num_classes=4, in_channels=1):
            self.num_samples = num_samples
            self.img_size = img_size
            self.num_classes = num_classes
            self.in_channels = in_channels
            
        def __len__(self):
            return self.num_samples
        
        def __getitem__(self, idx):
            np.random.seed(idx)
            img = np.random.randn(self.in_channels, self.img_size, self.img_size).astype(np.float32)
            mask = np.random.randint(0, self.num_classes, (self.img_size, self.img_size)).astype(np.int64)
            return torch.from_numpy(img), torch.from_numpy(mask)
    
    train_dataset = SyntheticMedicalDataset(500, config_data['img_size'], config_data['num_classes'], config_data['in_channels'])
    val_dataset = SyntheticMedicalDataset(100, config_data['img_size'], config_data['num_classes'], config_data['in_channels'])

# Create dataloaders
BATCH_SIZE = 4
NUM_CLASSES = config_data['num_classes']
IN_CHANNELS = config_data['in_channels']
IMG_SIZE = config_data['img_size']

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"\n✅ Dataset loaded!")
print(f"   Training samples: {len(train_dataset)}")
print(f"   Validation samples: {len(val_dataset)}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Number of classes: {NUM_CLASSES}")
print(f"   Input channels: {IN_CHANNELS}")

In [ ]:
# Visualize some training samples
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i in range(4):
    img, mask = train_dataset[i * 50]
    
    axes[0, i].imshow(img[0], cmap='gray')
    axes[0, i].set_title(f'Image {i+1}')
    axes[0, i].axis('off')
    
    axes[1, i].imshow(mask, cmap='viridis')
    axes[1, i].set_title(f'Mask {i+1}')
    axes[1, i].axis('off')

plt.suptitle('Training Samples', fontsize=14)
plt.tight_layout()
plt.show()

## 🎯 Training Configuration

In [ ]:
# Training configuration
config = {
    # Dataset (auto-configured from above)
    'dataset': DATASET,
    'in_channels': IN_CHANNELS,
    'num_classes': NUM_CLASSES,
    'img_size': IMG_SIZE,
    'class_names': config_data['class_names'],
    
    # Model
    'model': 'egm_net',           # 'egm_net', 'egm_net_lite', 'spectral_vmamba'
    'base_channels': 64,
    
    # Training
    'num_epochs': 100,
    'learning_rate': 1e-4,
    'weight_decay': 1e-5,
    'batch_size': BATCH_SIZE,
    
    # Loss weights
    'dice_weight': 1.0,
    'ce_weight': 1.0,
    'boundary_weight': 0.5,
    
    # Implicit representation
    'num_points': 2048,           # Points sampled for implicit loss
    'boundary_ratio': 0.5,        # Ratio of points near boundaries
    
    # Checkpointing
    'save_every': 10,
    'checkpoint_dir': f'./checkpoints_{DATASET}',
    
    # Early stopping
    'patience': 20,
}

# Create checkpoint directory
os.makedirs(config['checkpoint_dir'], exist_ok=True)

print("📋 Training Configuration:")
print(f"   Dataset: {config['dataset']}")
print(f"   Classes: {config['class_names']}")
print(f"   Model: {config['model']}")
print(f"   Epochs: {config['num_epochs']}")
print(f"   LR: {config['learning_rate']}")

In [ ]:
# Create model based on config
print(f"Creating {config['model']} model...")
print(f"   Input channels: {config['in_channels']}")
print(f"   Output classes: {config['num_classes']}")

if config['model'] == 'egm_net':
    model = EGMNet(
        in_channels=config['in_channels'],
        num_classes=config['num_classes'],
        img_size=config['img_size'],
        base_channels=config['base_channels'],
        num_stages=4,
        encoder_depth=2
    )
elif config['model'] == 'egm_net_lite':
    model = EGMNetLite(
        in_channels=config['in_channels'],
        num_classes=config['num_classes'],
        img_size=config['img_size']
    )
else:  # spectral_vmamba
    model = SpectralVMUNet(
        in_channels=config['in_channels'],
        out_channels=config['num_classes'],
        img_size=config['img_size'],
        base_channels=config['base_channels'],
        num_stages=4
    )

model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n✅ Model created!")
print(f"   Total parameters: {total_params:,} ({total_params/1e6:.2f}M)")
print(f"   Trainable parameters: {trainable_params:,}")

## 📈 Loss Functions & Metrics

In [ ]:
# Loss functions
class DiceLoss(nn.Module):
    """Dice loss for segmentation."""
    def __init__(self, smooth=1e-5):
        super().__init__()
        self.smooth = smooth
        
    def forward(self, pred, target):
        # pred: (B, C, H, W) logits
        # target: (B, H, W) class indices
        pred = F.softmax(pred, dim=1)
        num_classes = pred.shape[1]
        
        target_one_hot = F.one_hot(target, num_classes).permute(0, 3, 1, 2).float()
        
        intersection = (pred * target_one_hot).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))
        
        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        return 1.0 - dice.mean()


class BoundaryLoss(nn.Module):
    """Boundary-aware loss using Sobel edge detection."""
    def __init__(self):
        super().__init__()
        # Sobel kernels
        sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32)
        sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32)
        self.register_buffer('sobel_x', sobel_x.view(1, 1, 3, 3))
        self.register_buffer('sobel_y', sobel_y.view(1, 1, 3, 3))
        
    def get_boundaries(self, mask):
        # mask: (B, H, W)
        mask = mask.float().unsqueeze(1)
        edge_x = F.conv2d(mask, self.sobel_x, padding=1)
        edge_y = F.conv2d(mask, self.sobel_y, padding=1)
        edges = torch.sqrt(edge_x**2 + edge_y**2)
        return (edges > 0.5).float()
    
    def forward(self, pred, target):
        # Get boundary regions
        boundaries = self.get_boundaries(target)
        
        # Weight loss by boundary
        pred_probs = F.softmax(pred, dim=1)
        target_one_hot = F.one_hot(target, pred.shape[1]).permute(0, 3, 1, 2).float()
        
        # BCE at boundaries
        boundary_loss = F.binary_cross_entropy(
            pred_probs * boundaries, 
            target_one_hot * boundaries,
            reduction='sum'
        ) / (boundaries.sum() + 1e-6)
        
        return boundary_loss


class CombinedLoss(nn.Module):
    """Combined loss for EGM-Net training."""
    def __init__(self, dice_weight=1.0, ce_weight=1.0, boundary_weight=0.5, num_classes=2):
        super().__init__()
        self.dice_weight = dice_weight
        self.ce_weight = ce_weight
        self.boundary_weight = boundary_weight
        
        self.dice_loss = DiceLoss()
        self.ce_loss = nn.CrossEntropyLoss()
        self.boundary_loss = BoundaryLoss()
        
    def forward(self, outputs, targets):
        # For EGM-Net, outputs is a dict
        if isinstance(outputs, dict):
            pred = outputs['output']
            coarse = outputs.get('coarse')
            fine = outputs.get('fine')
            
            # Main loss
            loss = self.dice_weight * self.dice_loss(pred, targets)
            loss += self.ce_weight * self.ce_loss(pred, targets)
            loss += self.boundary_weight * self.boundary_loss(pred, targets)
            
            # Auxiliary losses (coarse and fine branches)
            if coarse is not None:
                loss += 0.3 * self.ce_loss(coarse, targets)
            if fine is not None:
                loss += 0.3 * self.ce_loss(fine, targets)
                
            return loss
        else:
            # Standard output (SpectralVMUNet)
            loss = self.dice_weight * self.dice_loss(outputs, targets)
            loss += self.ce_weight * self.ce_loss(outputs, targets)
            loss += self.boundary_weight * self.boundary_loss(outputs, targets)
            return loss


# Metrics
def compute_dice(pred, target, num_classes):
    """Compute per-class Dice scores."""
    dice_scores = []
    pred_classes = torch.argmax(pred, dim=1)
    
    for c in range(num_classes):
        pred_c = (pred_classes == c).float()
        target_c = (target == c).float()
        
        intersection = (pred_c * target_c).sum()
        union = pred_c.sum() + target_c.sum()
        
        if union > 0:
            dice = (2.0 * intersection) / union
        else:
            dice = torch.tensor(1.0)  # Both empty = perfect
            
        dice_scores.append(dice.item())
    
    return dice_scores


def compute_iou(pred, target, num_classes):
    """Compute per-class IoU scores."""
    iou_scores = []
    pred_classes = torch.argmax(pred, dim=1)
    
    for c in range(num_classes):
        pred_c = (pred_classes == c).float()
        target_c = (target == c).float()
        
        intersection = (pred_c * target_c).sum()
        union = pred_c.sum() + target_c.sum() - intersection
        
        if union > 0:
            iou = intersection / union
        else:
            iou = torch.tensor(1.0)
            
        iou_scores.append(iou.item())
    
    return iou_scores


print("✅ Loss functions and metrics defined!")

## 🚀 Training Loop

In [ ]:
# Full training function
def train_model(model, train_loader, val_loader, config, device):
    """Complete training loop with validation and checkpointing."""
    
    # Loss and optimizer
    criterion = CombinedLoss(
        dice_weight=config['dice_weight'],
        ce_weight=config['ce_weight'],
        boundary_weight=config['boundary_weight'],
        num_classes=config['num_classes']
    ).to(device)
    
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config['learning_rate'],
        weight_decay=config['weight_decay']
    )
    
    # Learning rate scheduler (cosine annealing)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=config['num_epochs'],
        eta_min=1e-6
    )
    
    # Training history
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_dice': [],
        'val_iou': [],
        'learning_rate': []
    }
    
    best_val_dice = 0.0
    patience_counter = 0
    
    print("\n" + "="*60)
    print("🚀 Starting Training")
    print("="*60)
    
    for epoch in range(config['num_epochs']):
        # =============== Training ===============
        model.train()
        train_loss = 0.0
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['num_epochs']} [Train]")
        
        for batch_idx, (images, masks) in enumerate(train_pbar):
            images = images.to(device)
            masks = masks.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            
            # Compute loss
            loss = criterion(outputs, masks)
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            train_loss += loss.item()
            train_pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        avg_train_loss = train_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)
        
        # =============== Validation ===============
        model.eval()
        val_loss = 0.0
        all_dice = []
        all_iou = []
        
        with torch.no_grad():
            for images, masks in tqdm(val_loader, desc=f"Epoch {epoch+1}/{config['num_epochs']} [Val]"):
                images = images.to(device)
                masks = masks.to(device)
                
                outputs = model(images)
                
                # Get prediction tensor
                pred = outputs['output'] if isinstance(outputs, dict) else outputs
                
                loss = criterion(outputs, masks)
                val_loss += loss.item()
                
                # Compute metrics
                dice_scores = compute_dice(pred, masks, config['num_classes'])
                iou_scores = compute_iou(pred, masks, config['num_classes'])
                
                all_dice.append(np.mean(dice_scores[1:]))  # Exclude background
                all_iou.append(np.mean(iou_scores[1:]))
        
        avg_val_loss = val_loss / len(val_loader)
        avg_val_dice = np.mean(all_dice)
        avg_val_iou = np.mean(all_iou)
        
        history['val_loss'].append(avg_val_loss)
        history['val_dice'].append(avg_val_dice)
        history['val_iou'].append(avg_val_iou)
        history['learning_rate'].append(optimizer.param_groups[0]['lr'])
        
        # Update learning rate
        scheduler.step()
        
        # Print epoch summary
        print(f"\n📊 Epoch {epoch+1}/{config['num_epochs']}")
        print(f"   Train Loss: {avg_train_loss:.4f}")
        print(f"   Val Loss:   {avg_val_loss:.4f}")
        print(f"   Val Dice:   {avg_val_dice:.4f}")
        print(f"   Val IoU:    {avg_val_iou:.4f}")
        print(f"   LR:         {optimizer.param_groups[0]['lr']:.6f}")
        
        # =============== Checkpointing ===============
        # Save best model
        if avg_val_dice > best_val_dice:
            best_val_dice = avg_val_dice
            patience_counter = 0
            
            checkpoint = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_val_dice': best_val_dice,
                'config': config,
                'history': history
            }
            torch.save(checkpoint, os.path.join(config['checkpoint_dir'], 'best_model.pth'))
            print(f"   ✅ New best model saved! (Dice: {best_val_dice:.4f})")
        else:
            patience_counter += 1
        
        # Save periodic checkpoints
        if (epoch + 1) % config['save_every'] == 0:
            checkpoint = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'history': history
            }
            torch.save(checkpoint, os.path.join(config['checkpoint_dir'], f'checkpoint_epoch_{epoch+1}.pth'))
        
        # Early stopping
        if patience_counter >= config['patience']:
            print(f"\n⚠️ Early stopping triggered after {epoch+1} epochs")
            break
    
    print("\n" + "="*60)
    print(f"🎉 Training completed!")
    print(f"   Best Val Dice: {best_val_dice:.4f}")
    print("="*60)
    
    return history


print("✅ Training function defined!")

In [ ]:
# 🚀 START TRAINING
history = train_model(model, train_loader, val_loader, config, device)

## 📊 Training Visualization

In [ ]:
# Plot training curves
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Loss curves
axes[0, 0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0, 0].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training & Validation Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Dice score
axes[0, 1].plot(history['val_dice'], label='Val Dice', linewidth=2, color='green')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Dice Score')
axes[0, 1].set_title('Validation Dice Score')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axhline(y=max(history['val_dice']), color='r', linestyle='--', alpha=0.5, label=f"Best: {max(history['val_dice']):.4f}")

# IoU score
axes[1, 0].plot(history['val_iou'], label='Val IoU', linewidth=2, color='orange')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('IoU Score')
axes[1, 0].set_title('Validation IoU Score')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Learning rate
axes[1, 1].plot(history['learning_rate'], label='Learning Rate', linewidth=2, color='purple')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Learning Rate')
axes[1, 1].set_title('Learning Rate Schedule')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_yscale('log')

plt.suptitle('Training Progress', fontsize=14)
plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📈 Training Summary:")
print(f"   Final Train Loss: {history['train_loss'][-1]:.4f}")
print(f"   Final Val Loss:   {history['val_loss'][-1]:.4f}")
print(f"   Best Val Dice:    {max(history['val_dice']):.4f}")
print(f"   Best Val IoU:     {max(history['val_iou']):.4f}")

## 🔍 Inference & Visualization

In [ ]:
# Load best model
checkpoint = torch.load(os.path.join(config['checkpoint_dir'], 'best_model.pth'))
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✅ Loaded best model from epoch {checkpoint['epoch']} (Dice: {checkpoint['best_val_dice']:.4f})")

In [ ]:
# Visualize predictions on validation set
model.eval()

fig, axes = plt.subplots(4, 4, figsize=(16, 16))

with torch.no_grad():
    for i in range(4):
        idx = i * (len(val_dataset) // 4)
        img, mask = val_dataset[idx]
        img = img.unsqueeze(0).to(device)
        
        # Get prediction
        outputs = model(img)
        pred = outputs['output'] if isinstance(outputs, dict) else outputs
        pred_mask = torch.argmax(pred, dim=1)[0].cpu()
        
        # Get energy map if available
        energy = outputs.get('energy', None)
        
        # Plot
        axes[i, 0].imshow(img[0, 0].cpu(), cmap='gray')
        axes[i, 0].set_title('Input Image')
        axes[i, 0].axis('off')
        
        axes[i, 1].imshow(mask, cmap='viridis')
        axes[i, 1].set_title('Ground Truth')
        axes[i, 1].axis('off')
        
        axes[i, 2].imshow(pred_mask, cmap='viridis')
        axes[i, 2].set_title('Prediction')
        axes[i, 2].axis('off')
        
        # Overlay
        overlay = img[0, 0].cpu().numpy()
        overlay = np.stack([overlay, overlay, overlay], axis=-1)
        pred_np = pred_mask.numpy()
        mask_np = mask.numpy()
        
        # Red for prediction, blue for ground truth
        overlay[..., 0] = np.where(pred_np > 0, 1.0, overlay[..., 0])
        overlay[..., 2] = np.where(mask_np > 0, 1.0, overlay[..., 2])
        overlay = np.clip(overlay, 0, 1)
        
        axes[i, 3].imshow(overlay)
        axes[i, 3].set_title('Overlay (Red=Pred, Blue=GT)')
        axes[i, 3].axis('off')

plt.suptitle('Validation Predictions', fontsize=14)
plt.tight_layout()
plt.savefig('predictions.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Visualize EGM-Net branches (Coarse vs Fine)
if config['model'] in ['egm_net', 'egm_net_lite']:
    fig, axes = plt.subplots(3, 5, figsize=(20, 12))
    
    with torch.no_grad():
        for i in range(3):
            idx = i * (len(val_dataset) // 3)
            img, mask = val_dataset[idx]
            img = img.unsqueeze(0).to(device)
            
            outputs = model(img)
            
            # Extract all outputs
            final_pred = torch.argmax(outputs['output'], dim=1)[0].cpu()
            coarse_pred = torch.argmax(outputs['coarse'], dim=1)[0].cpu()
            fine_pred = torch.argmax(outputs['fine'], dim=1)[0].cpu()
            energy = outputs['energy'][0, 0].cpu()
            
            # Plot
            axes[i, 0].imshow(img[0, 0].cpu(), cmap='gray')
            axes[i, 0].set_title('Input')
            axes[i, 0].axis('off')
            
            axes[i, 1].imshow(energy, cmap='hot')
            axes[i, 1].set_title('Energy Map')
            axes[i, 1].axis('off')
            
            axes[i, 2].imshow(coarse_pred, cmap='viridis')
            axes[i, 2].set_title('Coarse Branch')
            axes[i, 2].axis('off')
            
            axes[i, 3].imshow(fine_pred, cmap='viridis')
            axes[i, 3].set_title('Fine Branch')
            axes[i, 3].axis('off')
            
            axes[i, 4].imshow(final_pred, cmap='viridis')
            axes[i, 4].set_title('Final (Fused)')
            axes[i, 4].axis('off')
    
    plt.suptitle('EGM-Net Branch Analysis\\n(Energy-Gated Fusion of Coarse + Fine)', fontsize=14)
    plt.tight_layout()
    plt.savefig('branch_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()

## 🎯 Resolution-Free Inference Demo

In [ ]:
# Demonstrate resolution-free inference (unique to EGM-Net)
if config['model'] in ['egm_net', 'egm_net_lite']:
    print("🔬 Resolution-Free Inference Demo")
    print("   EGM-Net can render at ANY resolution without retraining!")
    
    resolutions = [64, 128, 256, 512]
    
    # Get a sample image
    img, mask = val_dataset[0]
    img = img.unsqueeze(0).to(device)
    
    fig, axes = plt.subplots(2, len(resolutions), figsize=(16, 8))
    
    with torch.no_grad():
        for i, res in enumerate(resolutions):
            # Render at different resolutions
            outputs = model(img, output_size=(res, res))
            pred = torch.argmax(outputs['output'], dim=1)[0].cpu()
            
            # Also show input at same res for comparison
            input_resized = F.interpolate(img, size=(res, res), mode='bilinear', align_corners=False)
            
            axes[0, i].imshow(input_resized[0, 0].cpu(), cmap='gray')
            axes[0, i].set_title(f'Input {res}×{res}')
            axes[0, i].axis('off')
            
            axes[1, i].imshow(pred, cmap='viridis')
            axes[1, i].set_title(f'Prediction {res}×{res}')
            axes[1, i].axis('off')
    
    plt.suptitle('Resolution-Free Rendering\\n(Same model weights, different output resolutions)', fontsize=14)
    plt.tight_layout()
    plt.savefig('resolution_free.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\\n✅ Same model can output at 64×64 to 512×512 (or higher)!")
    print("   This is impossible with standard CNN decoders.")

## 💾 Save & Export Model

In [ ]:
# Save final model to Google Drive
from google.colab import drive

try:
    drive.mount('/content/drive')
    
    # Save to Drive
    save_path = '/content/drive/MyDrive/EGM_Net_Models'
    os.makedirs(save_path, exist_ok=True)
    
    # Save checkpoint
    final_checkpoint = {
        'model_state_dict': model.state_dict(),
        'config': config,
        'history': history,
        'best_val_dice': max(history['val_dice'])
    }
    torch.save(final_checkpoint, os.path.join(save_path, 'egm_net_trained.pth'))
    
    # Also copy training curves
    import shutil
    shutil.copy('training_curves.png', save_path)
    shutil.copy('predictions.png', save_path)
    
    print(f"✅ Model saved to Google Drive: {save_path}")
    print(f"   - egm_net_trained.pth")
    print(f"   - training_curves.png")
    print(f"   - predictions.png")
    
except Exception as e:
    print(f"⚠️ Could not save to Google Drive: {e}")
    print("   Model is saved locally in ./checkpoints/")

## 📊 Final Evaluation Metrics

In [ ]:
# Final evaluation on full validation set
model.eval()

all_dice_scores = []
all_iou_scores = []
all_hd95_scores = []  # Hausdorff Distance 95

# Helper function for Hausdorff distance
def compute_hausdorff_95(pred, target):
    """Compute 95th percentile Hausdorff distance."""
    from scipy.ndimage import distance_transform_edt
    
    pred_np = pred.numpy().astype(bool)
    target_np = target.numpy().astype(bool)
    
    if pred_np.sum() == 0 or target_np.sum() == 0:
        return 0.0
    
    # Distance transforms
    pred_dist = distance_transform_edt(~pred_np)
    target_dist = distance_transform_edt(~target_np)
    
    # Get surface points
    pred_surface = pred_np & (distance_transform_edt(pred_np) <= 1)
    target_surface = target_np & (distance_transform_edt(target_np) <= 1)
    
    # Distances from pred surface to target, and vice versa
    d_pred_to_target = target_dist[pred_surface]
    d_target_to_pred = pred_dist[target_surface]
    
    if len(d_pred_to_target) == 0 or len(d_target_to_pred) == 0:
        return 0.0
    
    # 95th percentile
    hd95 = max(np.percentile(d_pred_to_target, 95), np.percentile(d_target_to_pred, 95))
    return hd95

print("🔍 Running final evaluation...")

with torch.no_grad():
    for images, masks in tqdm(val_loader, desc="Evaluating"):
        images = images.to(device)
        masks = masks.to(device)
        
        outputs = model(images)
        pred = outputs['output'] if isinstance(outputs, dict) else outputs
        pred_masks = torch.argmax(pred, dim=1)
        
        for b in range(images.shape[0]):
            # Per-sample metrics
            dice = compute_dice(pred[b:b+1], masks[b:b+1], config['num_classes'])
            iou = compute_iou(pred[b:b+1], masks[b:b+1], config['num_classes'])
            
            all_dice_scores.append(np.mean(dice[1:]))  # Exclude background
            all_iou_scores.append(np.mean(iou[1:]))
            
            # Hausdorff distance (for foreground)
            try:
                hd95 = compute_hausdorff_95(
                    (pred_masks[b] > 0).cpu(),
                    (masks[b] > 0).cpu()
                )
                all_hd95_scores.append(hd95)
            except:
                pass

# Print results
print("\n" + "="*60)
print("📊 FINAL EVALUATION RESULTS")
print("="*60)
print(f"\n{'Metric':<20} {'Mean':<12} {'Std':<12}")
print("-"*44)
print(f"{'Dice Score':<20} {np.mean(all_dice_scores):.4f}       {np.std(all_dice_scores):.4f}")
print(f"{'IoU Score':<20} {np.mean(all_iou_scores):.4f}       {np.std(all_iou_scores):.4f}")
if all_hd95_scores:
    print(f"{'HD95 (mm)':<20} {np.mean(all_hd95_scores):.2f}         {np.std(all_hd95_scores):.2f}")
print("-"*44)
print(f"\nTotal validation samples: {len(all_dice_scores)}")
print("="*60)

## 3. Test Monogenic Signal Processing

In [ ]:
# Create a test image with edges
def create_test_image(size=256):
    """Create synthetic medical-like image with organs."""
    img = torch.zeros(1, 1, size, size)
    
    # Add circular "organ"
    y, x = torch.meshgrid(torch.arange(size), torch.arange(size), indexing='ij')
    center1 = (size // 2, size // 2)
    radius1 = size // 4
    mask1 = ((x - center1[0])**2 + (y - center1[1])**2) < radius1**2
    img[0, 0, mask1] = 0.7
    
    # Add smaller "tumor"
    center2 = (size // 2 + 30, size // 2 - 20)
    radius2 = size // 10
    mask2 = ((x - center2[0])**2 + (y - center2[1])**2) < radius2**2
    img[0, 0, mask2] = 1.0
    
    # Add noise
    img = img + 0.05 * torch.randn_like(img)
    
    return img, mask1.float(), mask2.float()

# Create test image
test_img, organ_mask, tumor_mask = create_test_image(256)
print(f"Test image shape: {test_img.shape}")

In [ ]:
# Test Monogenic Energy Extraction
energy_extractor = EnergyMap(normalize=True, smoothing_sigma=1.0)
energy, mono_out = energy_extractor(test_img)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(test_img[0, 0], cmap='gray')
axes[0, 0].set_title('Input Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(energy[0, 0].detach(), cmap='hot')
axes[0, 1].set_title('Energy Map (Edges)')
axes[0, 1].axis('off')

axes[0, 2].imshow(mono_out['phase'][0, 0].detach(), cmap='twilight')
axes[0, 2].set_title('Phase')
axes[0, 2].axis('off')

axes[1, 0].imshow(mono_out['orientation'][0, 0].detach(), cmap='hsv')
axes[1, 0].set_title('Orientation')
axes[1, 0].axis('off')

axes[1, 1].imshow(mono_out['riesz_x'][0, 0].detach(), cmap='RdBu')
axes[1, 1].set_title('Riesz X Component')
axes[1, 1].axis('off')

axes[1, 2].imshow(mono_out['riesz_y'][0, 0].detach(), cmap='RdBu')
axes[1, 2].set_title('Riesz Y Component')
axes[1, 2].axis('off')

plt.suptitle('Monogenic Signal Decomposition', fontsize=14)
plt.tight_layout()
plt.show()

print("\n✅ Monogenic processing works correctly!")

## 4. Test Gabor Basis vs Fourier Features

In [ ]:
from gabor_implicit import GaborBasis, FourierFeatures

# Create coordinate grid
size = 128
y = torch.linspace(-1, 1, size)
x = torch.linspace(-1, 1, size)
yy, xx = torch.meshgrid(y, x, indexing='ij')
coords = torch.stack([xx, yy], dim=-1).view(1, -1, 2)  # (1, size*size, 2)

# Compare Gabor vs Fourier
gabor = GaborBasis(input_dim=2, num_frequencies=32)
fourier = FourierFeatures(input_dim=2, num_frequencies=32, scale=10.0)

gabor_features = gabor(coords)
fourier_features = fourier(coords)

print(f"Gabor features shape: {gabor_features.shape}")
print(f"Fourier features shape: {fourier_features.shape}")

In [ ]:
# Visualize first few basis functions
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i in range(4):
    # Gabor
    gabor_vis = gabor_features[0, :, i].view(size, size).detach().numpy()
    axes[0, i].imshow(gabor_vis, cmap='RdBu', vmin=-1, vmax=1)
    axes[0, i].set_title(f'Gabor Basis {i+1}')
    axes[0, i].axis('off')
    
    # Fourier
    fourier_vis = fourier_features[0, :, i].view(size, size).detach().numpy()
    axes[1, i].imshow(fourier_vis, cmap='RdBu', vmin=-1, vmax=1)
    axes[1, i].set_title(f'Fourier Basis {i+1}')
    axes[1, i].axis('off')

axes[0, 0].set_ylabel('Gabor\n(Localized)', fontsize=12)
axes[1, 0].set_ylabel('Fourier\n(Global)', fontsize=12)

plt.suptitle('Gabor vs Fourier Basis Functions\n(Gabor is localized → No Gibbs ringing)', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Create and Analyze Models

In [ ]:
# Create EGM-Net models
print("Creating models...")

# Full model
egm_net = EGMNet(
    in_channels=1,
    num_classes=3,
    img_size=256,
    base_channels=64,
    num_stages=4,
    encoder_depth=2
).to(device)

# Lite model
egm_lite = EGMNetLite(
    in_channels=1,
    num_classes=3,
    img_size=256
).to(device)

# Spectral Mamba (comparison)
spec_mamba = SpectralVMUNet(
    in_channels=1,
    out_channels=3,
    img_size=256,
    base_channels=64,
    num_stages=4
).to(device)

print("\n📊 Model Comparison:")
print("-" * 50)
models = {
    'EGM-Net Full': egm_net,
    'EGM-Net Lite': egm_lite,
    'SpectralVMUNet': spec_mamba
}

for name, model in models.items():
    params = sum(p.numel() for p in model.parameters())
    print(f"{name:20s}: {params:,} parameters ({params/1e6:.2f}M)")

## 6. Test Forward Pass

In [ ]:
# Test forward pass
test_input = torch.randn(2, 1, 256, 256).to(device)

print("Testing forward pass...")
print(f"Input shape: {test_input.shape}")

with torch.no_grad():
    # EGM-Net
    egm_out = egm_net(test_input)
    print(f"\n🔹 EGM-Net Output:")
    for k, v in egm_out.items():
        print(f"   {k}: {v.shape}")
    
    # SpectralVMUNet
    spec_out = spec_mamba(test_input)
    print(f"\n🔹 SpectralVMUNet Output: {spec_out.shape}")

print("\n✅ Forward pass successful!")

## 7. Test Resolution-Free Inference (Unique to EGM-Net)

In [ ]:
# EGM-Net can query at arbitrary coordinates!
print("Testing Resolution-Free Inference...")

# Create query points (random locations)
num_points = 10000
random_coords = torch.rand(1, num_points, 2).to(device) * 2 - 1  # [-1, 1]

with torch.no_grad():
    # Query at random points
    point_output = egm_net.query_points(test_input[:1], random_coords)
    
print(f"Query coordinates: {random_coords.shape}")
print(f"Point outputs: {point_output.shape}")
print("\n✅ Resolution-free inference works!")
print("   → You can zoom into boundaries at ANY resolution!")

In [ ]:
# Demonstrate resolution-free: render at different resolutions
resolutions = [64, 128, 256, 512]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

with torch.no_grad():
    for idx, res in enumerate(resolutions):
        # Render at this resolution
        output = egm_net(test_input[:1], output_size=(res, res))
        pred = torch.argmax(output['output'], dim=1)[0].cpu().numpy()
        
        axes[idx].imshow(pred, cmap='viridis')
        axes[idx].set_title(f'{res}×{res}')
        axes[idx].axis('off')

plt.suptitle('Resolution-Free Rendering (Same model, different output sizes)', fontsize=14)
plt.tight_layout()
plt.show()

## 8. Visualize Energy-Gated Fusion

In [ ]:
# Visualize the dual-branch architecture
with torch.no_grad():
    outputs = egm_net(test_input[:1])

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Input
axes[0, 0].imshow(test_input[0, 0].cpu(), cmap='gray')
axes[0, 0].set_title('Input Image')
axes[0, 0].axis('off')

# Energy Map
axes[0, 1].imshow(outputs['energy'][0, 0].cpu(), cmap='hot')
axes[0, 1].set_title('Energy Map (Edge Detection)')
axes[0, 1].axis('off')

# Coarse Branch
coarse_pred = torch.argmax(outputs['coarse'], dim=1)[0].cpu()
axes[0, 2].imshow(coarse_pred, cmap='viridis')
axes[0, 2].set_title('Coarse Branch (Smooth)')
axes[0, 2].axis('off')

# Fine Branch
fine_pred = torch.argmax(outputs['fine'], dim=1)[0].cpu()
axes[1, 0].imshow(fine_pred, cmap='viridis')
axes[1, 0].set_title('Fine Branch (Sharp)')
axes[1, 0].axis('off')

# Final Output
final_pred = torch.argmax(outputs['output'], dim=1)[0].cpu()
axes[1, 1].imshow(final_pred, cmap='viridis')
axes[1, 1].set_title('Final Output (Fused)')
axes[1, 1].axis('off')

# Difference
diff = (fine_pred != coarse_pred).float()
axes[1, 2].imshow(diff, cmap='Reds')
axes[1, 2].set_title('Difference (Fine vs Coarse)')
axes[1, 2].axis('off')

plt.suptitle('EGM-Net Dual-Branch Architecture', fontsize=14)
plt.tight_layout()
plt.show()

## 9. Quick Training Demo

In [ ]:
from train_egm import EGMNetTrainer, create_dummy_dataset
from torch.utils.data import DataLoader

# Create small dummy dataset
print("Creating dummy dataset...")
dataset = create_dummy_dataset(num_samples=16, img_size=256, num_classes=3)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Training config
config = {
    'learning_rate': 1e-4,
    'weight_decay': 1e-5,
    'num_epochs': 2,
    'num_points': 1024,
    'boundary_ratio': 0.5,
    'checkpoint_dir': './checkpoints_demo'
}

# Use lite model for faster training
model = EGMNetLite(in_channels=1, num_classes=3, img_size=256)
print(f"Model: {sum(p.numel() for p in model.parameters()):,} parameters")

In [ ]:
# Train for a few epochs
print("\nStarting training demo...")
trainer = EGMNetTrainer(model, config, device=device)
trainer.train(train_loader, num_epochs=2)

print("\n✅ Training demo completed!")

## 10. Inference Speed Benchmark

In [ ]:
import time

def benchmark_model(model, input_tensor, num_runs=50, warmup=10):
    """Benchmark inference speed."""
    model.eval()
    
    # Warmup
    with torch.no_grad():
        for _ in range(warmup):
            _ = model(input_tensor)
    
    if device == 'cuda':
        torch.cuda.synchronize()
    
    # Benchmark
    times = []
    with torch.no_grad():
        for _ in range(num_runs):
            start = time.time()
            _ = model(input_tensor)
            if device == 'cuda':
                torch.cuda.synchronize()
            times.append(time.time() - start)
    
    return np.mean(times) * 1000, np.std(times) * 1000  # ms

# Benchmark
print("Benchmarking inference speed...")
print("-" * 60)

test_input = torch.randn(1, 1, 256, 256).to(device)

for name, model in [('EGM-Net Full', egm_net), ('EGM-Net Lite', egm_lite)]:
    mean_time, std_time = benchmark_model(model, test_input)
    fps = 1000 / mean_time
    print(f"{name:20s}: {mean_time:.2f} ± {std_time:.2f} ms ({fps:.1f} FPS)")

print("\n✅ Benchmark completed!")

## 11. Summary

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════╗
║                    EGM-NET ARCHITECTURE SUMMARY                       ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                       ║
║  🔬 KEY INNOVATIONS:                                                  ║
║                                                                       ║
║  1. MONOGENIC ENERGY GATING                                          ║
║     • Physics-based edge detection (Riesz Transform)                 ║
║     • Automatically focuses on boundary regions                      ║
║     • Suppresses artifacts in flat regions                           ║
║                                                                       ║
║  2. GABOR BASIS (vs Fourier)                                         ║
║     • Localized oscillations (Gaussian × sin)                        ║
║     • NO Gibbs ringing artifacts                                     ║
║     • Sharp edges remain clean                                       ║
║                                                                       ║
║  3. DUAL-PATH ARCHITECTURE                                           ║
║     • Coarse Branch: Smooth body regions (Conv decoder)              ║
║     • Fine Branch: Sharp boundaries (Gabor Implicit)                 ║
║     • Energy-gated fusion: Best of both worlds                       ║
║                                                                       ║
║  4. RESOLUTION-FREE INFERENCE                                        ║
║     • Query at ANY coordinate → Infinite zoom                        ║
║     • No retraining needed for different resolutions                 ║
║     • Perfect for high-resolution medical imaging                    ║
║                                                                       ║
║  5. MAMBA ENCODER                                                    ║
║     • O(N) complexity (vs O(N²) for Transformers)                    ║
║     • Global context awareness                                       ║
║     • Efficient for large images                                     ║
║                                                                       ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                       ║
║  📊 MODEL SIZES:                                                      ║
║     • EGM-Net Full:  ~9.13M parameters                               ║
║     • EGM-Net Lite:  ~635K parameters                                ║
║     • SpectralVMUNet: ~10.31M parameters                             ║
║                                                                       ║
╚══════════════════════════════════════════════════════════════════════╝
""")

---

## 📚 Next Steps

1. **Train on real data**: Replace dummy dataset with medical imaging dataset (e.g., Synapse, ACDC)
2. **Tune hyperparameters**: Adjust `num_frequencies`, `boundary_ratio`, learning rate
3. **Evaluate metrics**: Dice score, IoU, Hausdorff distance
4. **Ablation study**: Compare Gabor vs Fourier, with/without energy gating

---

**Repository**: https://github.com/QuocKhanhLuong/FourierNetwork